<a href="https://colab.research.google.com/github/AndreasPB/ai-stuff/blob/master/bad_chat_gpt_clone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building a bad Generatively Pretrained Transformer (GPT) by following https://youtu.be/kCc8FmEb1nYs

In [48]:
# Read some txt dataset
with open('WikiQA-train.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [49]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  3668541


In [50]:
# Let's look at the first 1000 characters
print(text[:1000])

how are glacier caves formed ?	A partly submerged glacier cave on Perito Moreno Glacier .	0
how are glacier caves formed ?	The ice facade is approximately 60 m high	0
how are glacier caves formed ?	Ice formations in the Titlis glacier cave	0
how are glacier caves formed ?	A glacier cave is a cave formed within the ice of a glacier .	1
how are glacier caves formed ?	Glacier caves are often called ice caves , but this term is properly used to describe bedrock caves that contain year-round ice .	0
How are the directions of the velocity and force vectors related in a circular motion	In physics , circular motion is a movement of an object along the circumference of a circle or rotation along a circular path .	0
How are the directions of the velocity and force vectors related in a circular motion	It can be uniform , with constant angular rate of rotation ( and constant speed ) , or non-uniform with a changing rate of rotation .	0
How are the directions of the velocity and force vectors relat

In [51]:
# Here are all the unique characters that occur in this text
chars = sorted(set(text))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)

	
 !#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]_`abcdefghijklmnopqrstuvwxyz{|}~¢£§­®°±·ÁÉ×ØàáâãäæçèéêëíïðñóôöøùúûüĀāąćčĐēīłňōŚşšūƎưǎɔɪʌʔʻʾʿˀˈάέήίαβγεζηθικλμνξοπρςστυφχψωόύώАаклсяאבדהוזחטילםמןקרשरशी्ชมยรวศาิีเምሴḥṭạảấễệốồộờủỹἀἈἐἱἸὁὐὰὶὸὺ‐–—‘’“”†•‰′⁄€−≈≤≥┌┐★♀♂♋♙♟がなひらカタナ一劉勝圖安小李萬言龍원학
294


In [52]:
stoi: dict[str, int] = { ch: i for i, ch in enumerate(chars) }
itos: dict[int, str] = { i: ch for i, ch in enumerate(chars) }

def encode(s: str) -> list[int]:
    return [stoi[c] for c in s]

def decode(l: list) -> str:
    return "".join([itos[i] for i in l])

sentence = "haha lol"
print("encoded:", encode(sentence))
print("decoded:", decode(encode(sentence)))

encoded: [72, 65, 72, 65, 2, 76, 79, 76]
decoded: haha lol


In [53]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([3668541]) torch.int64
tensor([72, 79, 87,  2, 65, 82, 69,  2, 71, 76, 65, 67, 73, 69, 82,  2, 67, 65,
        86, 69, 83,  2, 70, 79, 82, 77, 69, 68,  2, 32,  0, 34,  2, 80, 65, 82,
        84, 76, 89,  2, 83, 85, 66, 77, 69, 82, 71, 69, 68,  2, 71, 76, 65, 67,
        73, 69, 82,  2, 67, 65, 86, 69,  2, 79, 78,  2, 49, 69, 82, 73, 84, 79,
         2, 46, 79, 82, 69, 78, 79,  2, 40, 76, 65, 67, 73, 69, 82,  2, 15,  0,
        17,  1, 72, 79, 87,  2, 65, 82, 69,  2])


In [54]:
# Split it up
n = int(0.9 * len(data))

train_data = data[:n]
val_data = data[n:]

In [55]:
block_size = 8
train_data[:block_size + 1]

tensor([72, 79, 87,  2, 65, 82, 69,  2, 71])

In [56]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]
for t in range(block_size):
    context = x[:t + 1]
    target = y[t]
    print(f"When input context: {context}, the target is: {target}")

When input context: tensor([72]), the target is: 79
When input context: tensor([72, 79]), the target is: 87
When input context: tensor([72, 79, 87]), the target is: 2
When input context: tensor([72, 79, 87,  2]), the target is: 65
When input context: tensor([72, 79, 87,  2, 65]), the target is: 82
When input context: tensor([72, 79, 87,  2, 65, 82]), the target is: 69
When input context: tensor([72, 79, 87,  2, 65, 82, 69]), the target is: 2
When input context: tensor([72, 79, 87,  2, 65, 82, 69,  2]), the target is: 71


In [64]:
from torch.functional import Tensor

batch_size = 4  # Amount of parallel processes
block_size = 4  # Max context length for a prediction

def get_batch(split: str) -> tuple[Tensor, Tensor]:
    # Generate a small batch of data (inputs x and targets y)

    data = train_data if split == "train" else val_data

    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i + block_size] for i in ix])
    y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])

    return x, y

xb, yb = get_batch("train")
print(f"inputs: \n{xb.shape} \n{xb}")
print(f"targets: \n{yb.shape} \n{yb}")

print("\n-----\n")

for b in range(batch_size):  # Batch dimension
    for t in range(block_size):  # Time dimension
        context = xb[b, :t + 1]
        target = yb[b, t]
        print(f"When input is: {context.tolist()}, the target is: {target}")

inputs: 
torch.Size([4, 4]) 
tensor([[76, 65, 84, 73],
        [65, 84, 69, 68],
        [78,  2, 76, 73],
        [ 2, 83, 89, 83]])
targets: 
torch.Size([4, 4]) 
tensor([[65, 84, 73, 86],
        [84, 69, 68,  2],
        [ 2, 76, 73, 66],
        [83, 89, 83, 84]])

-----

When input is: [76], the target is: 65
When input is: [76, 65], the target is: 84
When input is: [76, 65, 84], the target is: 73
When input is: [76, 65, 84, 73], the target is: 86
When input is: [65], the target is: 84
When input is: [65, 84], the target is: 69
When input is: [65, 84, 69], the target is: 68
When input is: [65, 84, 69, 68], the target is: 2
When input is: [78], the target is: 2
When input is: [78, 2], the target is: 76
When input is: [78, 2, 76], the target is: 73
When input is: [78, 2, 76, 73], the target is: 66
When input is: [2], the target is: 83
When input is: [2, 83], the target is: 89
When input is: [2, 83, 89], the target is: 83
When input is: [2, 83, 89, 83], the target is: 84
